In [1]:
import numpy as np
print(np.__version__)
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from datetime import datetime
import os
import random
from shutil import copyfile
from sklearn.model_selection import train_test_split
assert torch.cuda.is_available(), "Missing CUDA"

1.21.2


In [2]:
torch.version.cuda

'10.2'

In [3]:
torch.cuda.device_count()

1

In [4]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28)

# Set Seeds for Reproducibility

In [5]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Get Data

In [6]:
from datasets import LoadDataset, CustomOutput
from datasets.custom_output import image_tensor, bounding_boxes, float_mask
from trafo.randomize.default_augmentation import default_augmentation_only_geometric

loaded_data = LoadDataset("_data/preprocessed256_new", image_dtype=float,
                          label_dtype=float)

dataset_plain = CustomOutput(loaded_data, image_tensor, float_mask)
dataset_aug = CustomOutput(loaded_data, image_tensor, bounding_boxes, trafo=default_augmentation_only_geometric)

In [7]:
len(dataset_plain)

6334

In [8]:
"""i_img = 42
example_img = dataset_aug[i_img]

fig, ax = plt.subplots(1,2)
plt.gray()
ax[0].imshow(example_img[0][0])
ax[1].imshow(example_img[1][0])
plt.show()"""

'i_img = 42\nexample_img = dataset_aug[i_img]\n\nfig, ax = plt.subplots(1,2)\nplt.gray()\nax[0].imshow(example_img[0][0])\nax[1].imshow(example_img[1][0])\nplt.show()'

In [9]:
# dataset_aug[42][1].shape

In [10]:
"""batch_size = 16

# get good split of dataset -> dividable by batch_size
l = len(dataset_aug)
indices = list(range(l))
train_size = (l // (batch_size * 6)) * batch_size * 5
val_size = l - train_size
print("Training: ", train_size, "Validation: ", val_size)
train_indices, val_indices = train_test_split(indices, random_state=4, train_size=train_size, test_size=val_size)

train_set = Subset(dataset_aug, train_indices)
val_set = Subset(dataset_plain, val_indices)"""

"""dataset = dataset_aug
batch_size = 16
l = len(dataset)
x = l // (batch_size *6 )
split = [x * batch_size * 5, l - x * batch_size * 5]
print(split)
train_set, val_set = torch.utils.data.random_split(dataset, split)""";

In [11]:
"""# alternative:
batch_size = 16
l = len(dataset_aug)
x = l // (batch_size *6 )
split = [x * batch_size * 5, l - x * batch_size * 5]
print(split)
train_set, val_set = torch.utils.data.random_split(dataset_aug, split)"""

'# alternative:\nbatch_size = 16\nl = len(dataset_aug)\nx = l // (batch_size *6 )\nsplit = [x * batch_size * 5, l - x * batch_size * 5]\nprint(split)\ntrain_set, val_set = torch.utils.data.random_split(dataset_aug, split)'

In [12]:
batch_size = 16

# get good split of dataset -> dividable by batch_size
l = len(dataset_aug)
indices = list(range(l))
train_size = (l // (batch_size * 6)) * batch_size * 5
val_size = l - train_size
print("Training: ", train_size, "Validation: ", val_size)
train_indices, val_indices = train_test_split(indices, random_state=4, train_size=train_size, test_size=val_size)

train_set = Subset(dataset_aug, train_indices)
val_set = Subset(dataset_plain, val_indices)

Training:  5200 Validation:  1134


In [13]:
dataloader_train = DataLoader(train_set, batch_size=batch_size,
                        shuffle=True, num_workers=0)#, pin_memory = True)
dataloader_val = DataLoader(val_set, batch_size=batch_size,
                        shuffle=True, num_workers=0, pin_memory = True)

In [14]:
"""for x,y in dataloader_val:
    fig, ax = plt.subplots(1,2)
    plt.gray()
    ax[0].imshow(x[0][0])
    ax[1].imshow(y[0][0])
    plt.show()
    break"""

'for x,y in dataloader_val:\n    fig, ax = plt.subplots(1,2)\n    plt.gray()\n    ax[0].imshow(x[0][0])\n    ax[1].imshow(y[0][0])\n    plt.show()\n    break'

# Get Network

In [15]:
from network.unet import Unet

# Get Training

In [16]:
from network.Model import OurModel
from network.losses import DiceLoss, BCEandDiceLoss

In [17]:
import torch.nn as nn
#criterion = nn.CrossEntropyLoss().cuda()
#criterion = DiceLoss() # use binary cross entropy loss!
criterion = BCEandDiceLoss()
#criterion = nn.BCELoss().cuda()
network = Unet()
path = f"./_trainings/{datetime.now().strftime('%d-%m_%H-%M')}"
if os.path.exists(path):
    print("PATH already exists")
else:
    print(f"Make {path} directory")
    os.makedirs(path)

Make ./_trainings/16-09_17-03 directory


In [18]:
Model = OurModel(name = "unet", network=network, criterion=criterion, path_dir=path, lr=0.001,
                 batch_size = batch_size, verbose = True, segmentation=True, data_trafo = dataset_aug.trafo)
# save a json file which indicates what parameters are used for training
Model.save_configuration()

In [19]:
%%javascript
// copy the current version of the jupyter-notebook in the chosen directory (for documentation of the used code and
// setting...)
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [20]:
nb_name

'TrainingUnet-Copy2.ipynb'

In [21]:
copyfile(os.path.join(os.getcwd(), nb_name), os.path.join(path, nb_name))

'./_trainings/16-09_17-03/TrainingUnet-Copy2.ipynb'

In [22]:
Model.train(100, dataloader_train, validate = True,
            dataloader_val=dataloader_val, save_observables = True)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/325 [00:00<?, ?it/s]

/home/students/patzig/aml37/.venv/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch1: training_loss = 1.3159095155275784
epoch1: training_dice = 0.3155899330687064


  0%|          | 0/71 [00:00<?, ?it/s]

...

epoch5: training_loss = 1.2776105807377742
epoch5: training_dice = 0.32298037926854495


  0%|          | 0/71 [00:00<?, ?it/s]

epoch5: validation_loss = 1.2747130242871567
epoch5: validation_dice = 0.3210244745435849


  0%|          | 0/325 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#Model.load_weights(f"/Unet_first_try_e50.ckpt")

In [ ]:
"""for x,y in dataloader_val:
    Model.network.eval()
    y_hat = Model.network(x.float().cuda())
    break

y_hat = y_hat.cpu()
y_hat = y_hat.detach().numpy()
y = y.numpy()
fig, ax = plt.subplots(len(y_hat),3, figsize=(10, 100))
for i in range(len(y_hat)):
    ax[i,0].imshow(y_hat[i])
    ax[i,1].imshow(np.round(y_hat[i]))
    ax[i,2].imshow(y[i])
"""